In [2]:
import cv2 as cv
import tkinter as tk
import numpy as np
from mtcnn import MTCNN

In [4]:
from retinaface import RetinaFace

In [45]:
img1=cv.imread('../Data/Images/cat (1).jpeg',cv.IMREAD_REDUCED_COLOR_2)
img2=cv.imread('../Data/Images/cat (2).jpeg',cv.IMREAD_REDUCED_COLOR_2)
img3=cv.imread('../Data/Images/cat (3).jpeg',cv.IMREAD_REDUCED_COLOR_2)

In [33]:
def show(img,name=None):
    window_name=name if name else "Default"
    cv.imshow(window_name,img)
    cv.waitKey()
    cv.destroyAllWindows()

In [46]:
show(img1)

In [35]:
cv.imwrite('catcute.jpeg',img1)

True

Differnet image loading

In [36]:

# 1. Load a color image (ignores alpha channel)
img_color = cv.imread("../Data/Images/cat (1).jpeg", cv.IMREAD_COLOR)        # or cv.imread("../Data/Images/cat (1).jpeg", 1)

# 2. Load a grayscale image
img_gray = cv.imread("../Data/Images/cat (1).jpeg", cv.IMREAD_GRAYSCALE)     # or cv.imread("../Data/Images/cat (1).jpeg", 0)

# 3. Load the image as-is (including alpha channel if present)
img_unchanged = cv.imread("../Data/Images/cat (1).jpeg", cv.IMREAD_UNCHANGED)  # or cv.imread("../Data/Images/cat (1).jpeg", -1)

# 4. Load image preserving bit depth
img_anydepth = cv.imread("../Data/Images/cat (1).jpeg", cv.IMREAD_ANYDEPTH)   # 16-bit, 32-bit images

# 5. Load image in any color format
img_anycolor = cv.imread("../Data/Images/cat (1).jpeg", cv.IMREAD_ANYCOLOR)

# 6. Load reduced-size grayscale images
img_gray_half = cv.imread("../Data/Images/cat (1).jpeg", cv.IMREAD_REDUCED_GRAYSCALE_2)
img_gray_quarter = cv.imread("../Data/Images/cat (1).jpeg", cv.IMREAD_REDUCED_GRAYSCALE_4)
img_gray_eighth = cv.imread("../Data/Images/cat (1).jpeg", cv.IMREAD_REDUCED_GRAYSCALE_8)

# 7. Load reduced-size color images
img_color_half = cv.imread("../Data/Images/cat (1).jpeg", cv.IMREAD_REDUCED_COLOR_2)
img_color_quarter = cv.imread("../Data/Images/cat (1).jpeg", cv.IMREAD_REDUCED_COLOR_4)
img_color_eighth = cv.imread("../Data/Images/cat (1).jpeg", cv.IMREAD_REDUCED_COLOR_8)

In [24]:
show(img_color_half)

In [48]:
show(img2)

In [47]:
show(img3)

In [52]:
def play(video_path, skip_frames=1):
    """
    Plays a video scaled to fit your screen with correct FPS.
    
    :param video_path: Path to the video file
    :param skip_frames: Show every Nth frame (1 = show all)
    """
    # Get screen resolution using Tkinter
    root = tk.Tk()
    screen_width = root.winfo_screenwidth()
    screen_height = root.winfo_screenheight()
    root.destroy()

    cap = cv.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error opening video file")
        return

    fps = cap.get(cv.CAP_PROP_FPS)
    total_frames = int(cap.get(cv.CAP_PROP_FRAME_COUNT))
    print(f"FPS: {fps}, Total frames: {total_frames}, Screen: {screen_width}x{screen_height}")

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Skip frames if needed
        current_frame = int(cap.get(cv.CAP_PROP_POS_FRAMES))
        if current_frame % skip_frames != 0:
            continue

        # Resize frame to fit screen
        h, w = frame.shape[:2]
        scale = min(screen_width / w, screen_height / h, 1.0)
        if scale < 1.0:
            frame = cv.resize(frame, (int(w * scale), int(h * scale)))

        cv.imshow("Video", frame)
        if cv.waitKey(int(1000 / fps)) & 0xFF == ord('q'):
            break

    cap.release()
    cv.destroyAllWindows()

Cool kernels exploration

In [62]:
# Define filters with descriptive info
filters = {
    "Blur": {
        "func": lambda x: cv.blur(x, (5,5)),
        "desc": "Simple average blur (5x5 kernel)"
    },
    "GaussianBlur": {
        "func": lambda x: cv.GaussianBlur(x, (5,5), 0),
        "desc": "Gaussian blur (5x5 kernel, sigma auto)"
    },
    "MedianBlur": {
        "func": lambda x: cv.medianBlur(x, 5),
        "desc": "Median blur (removes salt-and-pepper noise)"
    },
    "BilateralFilter": {
        "func": lambda x: cv.bilateralFilter(x, 9, 75, 75),
        "desc": "Edge-preserving bilateral filter"
    },
    "SobelX": {
        "func": lambda x: cv.Sobel(x, cv.CV_64F, 1, 0, ksize=3),
        "desc": "Sobel gradient in X direction (vertical edges)"
    },
    "SobelY": {
        "func": lambda x: cv.Sobel(x, cv.CV_64F, 0, 1, ksize=3),
        "desc": "Sobel gradient in Y direction (horizontal edges)"
    },
    "Laplacian": {
        "func": lambda x: cv.Laplacian(x, cv.CV_64F),
        "desc": "Laplacian (second-order derivative, detects all edges)"
    },
    "Canny": {
        "func": lambda x: cv.Canny(x, 100, 200),
        "desc": "Canny edge detector (binary edges)"
    }
}
other_filters = {
    "DetailEnhance": {
        "func": lambda x: cv.detailEnhance(x, sigma_s=10, sigma_r=0.15),
        "desc": "Enhances details and local contrast for a punchy look"
    },
    "Stylization": {
        "func": lambda x: cv.stylization(x, sigma_s=150, sigma_r=0.25),
        "desc": "Turns image into a cartoon/painterly style"
    },
    "PencilSketchGray": {
        "func": lambda x: cv.pencilSketch(x, sigma_s=60, sigma_r=0.07, shade_factor=0.05)[0],
        "desc": "Grayscale pencil sketch effect"
    },
    "PencilSketchColor": {
        "func": lambda x: cv.pencilSketch(x, sigma_s=60, sigma_r=0.07, shade_factor=0.05)[1],
        "desc": "Colored pencil sketch effect"
    },
    "Erosion": {
        "func": lambda x: cv.erode(cv.cvtColor(x, cv.COLOR_BGR2GRAY), np.ones((3,3), np.uint8), iterations=1),
        "desc": "Erodes bright regions (shrinks shapes)"
    },
    "Dilation": {
        "func": lambda x: cv.dilate(cv.cvtColor(x, cv.COLOR_BGR2GRAY), np.ones((3,3), np.uint8), iterations=1),
        "desc": "Dilates bright regions (expands shapes)"
    },
    "Opening": {
        "func": lambda x: cv.morphologyEx(cv.cvtColor(x, cv.COLOR_BGR2GRAY), cv.MORPH_OPEN, np.ones((3,3), np.uint8)),
        "desc": "Erosion followed by dilation (removes small noise)"
    },
    "Closing": {
        "func": lambda x: cv.morphologyEx(cv.cvtColor(x, cv.COLOR_BGR2GRAY), cv.MORPH_CLOSE, np.ones((3,3), np.uint8)),
        "desc": "Dilation followed by erosion (fills small holes)"
    },
    "MorphGradient": {
        "func": lambda x: cv.morphologyEx(cv.cvtColor(x, cv.COLOR_BGR2GRAY), cv.MORPH_GRADIENT, np.ones((3,3), np.uint8)),
        "desc": "Difference between dilation and erosion (highlights object boundaries)"
    },
    "AdaptiveThresholdMean": {
        "func": lambda x: cv.adaptiveThreshold(cv.cvtColor(x, cv.COLOR_BGR2GRAY), 255, cv.ADAPTIVE_THRESH_MEAN_C, cv.THRESH_BINARY, 11, 2),
        "desc": "Adaptive threshold using neighborhood mean"
    },
    "AdaptiveThresholdGaussian": {
        "func": lambda x: cv.adaptiveThreshold(cv.cvtColor(x, cv.COLOR_BGR2GRAY), 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY, 11, 2),
        "desc": "Adaptive threshold using Gaussian-weighted sum"
    },
    "OtsuThreshold": {
        "func": lambda x: cv.threshold(cv.cvtColor(x, cv.COLOR_BGR2GRAY), 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)[1],
        "desc": "Automatically computes optimal global threshold"
    }
}

def filter(img):
    for name, info in other_filters.items():
        filtered = info["func"](img)
        
        # Convert float images to uint8 for display
        if filtered.dtype != np.uint8:
            filtered = cv.convertScaleAbs(filtered)
        
        # Show image with descriptive window title
        window_title = f"{info['desc']}"
        cv.imshow(window_title, filtered)
        cv.waitKey(0)
    
    cv.destroyAllWindows()



In [66]:
filter(img3)

In [69]:
def detect_features(img):
    """
    img: already-loaded image (BGR) as a NumPy array
    """
    if img is None:
        print("Image not found!")
        return

    # Convert to grayscale (most detectors require this)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    # Define feature detectors
    detectors = {
        "SIFT": cv.SIFT_create(),
        "ORB": cv.ORB_create(),
        "BRISK": cv.BRISK_create(),
        "AKAZE": cv.AKAZE_create(),
        "FAST": cv.FastFeatureDetector_create(),
        "KAZE": cv.KAZE_create()
    }

    for name, detector in detectors.items():
        if detector is None:
            continue  # Skip if detector not available
        if name in ["FAST"]:
            # FAST only detects keypoints, no descriptors
            keypoints = detector.detect(gray, None)
        else:
            keypoints, _ = detector.detectAndCompute(gray, None)

        # Draw keypoints
        img_kp = cv.drawKeypoints(img, keypoints, None, flags=cv.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

        # Show result
        cv.imshow(f"{name} Keypoints - Total: {len(keypoints)}", img_kp)
        cv.waitKey(0)

    cv.destroyAllWindows()


In [70]:
detect_features(img1) 